In [1]:
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("libgsl.dylib")
#pragma cling load("libG4processes.dylib")
#include <iostream>
#include "G4LegendrePolynomial.hh"
#include <random>
#include <cstdlib>
#include "gsl/gsl_sf_legendre.h"
#include <stdio.h> 
#include <stdlib.h> 
#include <chrono>

#include <iomanip>

{
    G4LegendrePolynomial lp;
    gsl_sf_legendre_t norm = GSL_SF_LEGENDRE_NONE;
    std::random_device rd;
    std::mt19937 mt(rd());
    std::uniform_real_distribution<double> xdist(-1, 1);
    
    std::cout << "L,  " << " t_gsl_s, " << "t_g4_s, " << "diff2" << std::endl;    
    

       for (int l=0; l < 10; ++l){
           double gsl_acc_time_ns = 0;
           double g4_acc_time_ns = 0;
           double sum_diff2 = 0;
           
           std::vector<double> result_array(gsl_sf_legendre_array_n(l), 0);
           
           
           for (int i=0; i < 1000000; ++i){
                double x = xdist(mt);
                
                auto started_gsl = std::chrono::high_resolution_clock::now();
                gsl_sf_legendre_array_e(norm, l, x, -1, &(result_array[0]));
                auto done_gsl = std::chrono::high_resolution_clock::now(); 
                 
                gsl_acc_time_ns += std::chrono::duration_cast<std::chrono::nanoseconds>(done_gsl-started_gsl).count();
                
                for (int m = 0; m <= l; ++m){
                    auto started_g4 = std::chrono::high_resolution_clock::now();
                    double alpg4 = lp.EvalAssocLegendrePoly(l, m, x); 
                    auto done_g4 = std::chrono::high_resolution_clock::now(); 
                    g4_acc_time_ns += std::chrono::duration_cast<std::chrono::nanoseconds>(done_g4-started_g4).count(); 
                    
               
                    double alp_gsl = result_array[gsl_sf_legendre_array_index(l, m)] ;
                    sum_diff2 += pow(alp_gsl - alpg4, 2);
                      
               } // loop over m
           } // loop over i
     
            std::cout << l << " , " <<  gsl_acc_time_ns * 1e-9 << " , " << g4_acc_time_ns * 1e-9 << " , " <<   sum_diff2 * 1e-18 << std::endl;
            
       } // loop over l

}
   

L,   t_gsl_s, t_g4_s, diff2
0 , 0.0402801 , 0.0297562 , 0
1 , 0.0467552 , 0.0690549 , 3.17018e-45
2 , 0.0518691 , 0.10286 , 1.14435e-43
3 , 0.0661832 , 0.140879 , 3.48413e-42
4 , 0.0784612 , 0.199306 , 2.46253e-40
5 , 0.112947 , 0.438049 , 4.05538e-36
6 , 0.147027 , 0.690096 , 1.82395e-34
7 , 0.180548 , 1.09393 , 1.64465e-32
8 , 0.206648 , 1.60344 , 5.41531e-31
9 , 0.267079 , 2.66795 , 1.6964e-27
